In [1]:
import tensorflow as tf

In [86]:
D = 4
t = -0.0004

In [87]:
with tf.name_scope("constant"):
    H = tf.reshape(tf.constant([[0.25,0,0,0],[0,-0.25,0.5,0],[0,0.5,-0.25,0],[0,0,0,0.25]]),[2,2,2,2],name="Hamiltonian")
    I = tf.reshape(tf.constant([[1.,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]]),[2,2,2,2],"Identity")
    expH = I + t * H
    Z = tf.zeros(D,name="Zeros")

In [88]:
with tf.name_scope("variables"):
    A = tf.Variable(tf.random_uniform([D, D, 2], minval=-1., maxval=1.),name="A")
    B = tf.Variable(tf.random_uniform([D, D, 2], minval=-1., maxval=1.),name="B")
    EAB = tf.Variable(tf.ones(D), name="EAB")
    EBA = tf.Variable(tf.ones(D), name="EBA")
    L = tf.Variable(tf.random_uniform([D, D], minval=-1., maxval=1.),name="L")
    R = tf.Variable(tf.random_uniform([D, D], minval=-1., maxval=1.),name="R")
    init = tf.global_variables_initializer()

In [89]:
def update(A,B,EAB,EBA,s):
    label = "%s_H_%s"%(s[0],s[1])
    with tf.name_scope(label):
        EBA_A = tf.multiply(tf.reshape(EBA,[D,1,1]),A,name="%s_%s"%(s[3],s[0]))
        EBA_A_EAB = tf.multiply(EBA_A,tf.reshape(EAB*EAB,[1,D,1]),name="%s_%s_%s"%(s[3],s[0],s[2]))
        B_EBA = tf.multiply(B,tf.reshape(EBA,[1,D,1]),name="%s_%s"%(s[1],s[3]))
        A_H = tf.tensordot(EBA_A_EAB,expH,[[2],[0]],name="%s_H"%s[0])
        BIG = tf.tensordot(A_H,B_EBA,[[1,2],[0,2]],name=label) # D 2 2 D
        S, U, V = tf.svd(tf.reshape(BIG,[2*D,2*D]),name="%s_SVD"%label) # D 2 2D, 2D, 2D 2 D
        for p in xrange(4):
            tf.summary.scalar("s%d"%p,S[p])
        nEAB = tf.sqrt(tf.slice(S,[0],[D],name="n%s"%s[2])) #s[:D]
        pre_reci = tf.reciprocal(EBA,name="pre_reci") # D
        reci = tf.where(tf.is_inf(pre_reci),Z,pre_reci,name="%s_Eviron_Reci"%label) # D
        nA = tf.transpose(tf.multiply(tf.reshape(U[:,:D],[D,2,D]),tf.reshape(reci,[D,1,1])),[0,2,1],name="n%s"%s[0])
        nB = tf.transpose(tf.multiply(tf.reshape(V[:D,:],[D,2,D]),tf.reshape(reci,[1,1,D])),[0,2,1],name="n%s"%s[1])
        return tf.assign(A,nA/tf.norm(nA)),tf.assign(B,nB/tf.norm(nB)),tf.assign(EAB,nEAB/tf.norm(nEAB))

In [90]:
updateAB = update(A,B,EAB,EBA,["A","B","EAB","EBA"])
updateBA = update(B,A,EBA,EAB,["B","A","EBA","EAB"])

In [91]:
with tf.name_scope("updateR"):
    RB = tf.tensordot(tf.tensordot(R,B,[[1],[1]]),B,[[0,2],[1,2]],name="RB")
    RBE = tf.multiply(tf.multiply(RB,tf.reshape(EAB*EAB,[D,1])),tf.reshape(EAB*EAB,[1,D]),name="RBE")
    RBEA = tf.tensordot(tf.tensordot(RBE/tf.norm(RBE),A,[[1],[1]]),A,[[0,2],[1,2]],name="RBEA")
    RBEAE = tf.multiply(tf.multiply(RBEA,tf.reshape(EBA*EBA,[D,1])),tf.reshape(EBA*EBA,[1,D]),name="RBEAE")
    updateR = tf.assign(R,RBEAE/tf.norm(RBEAE))

with tf.name_scope("updateL"):
    LA = tf.tensordot(tf.tensordot(L,A,[[1],[0]]),A,[[0,2],[0,2]],name="LA")
    LAE = tf.multiply(tf.multiply(LA,tf.reshape(EAB*EAB,[D,1])),tf.reshape(EAB*EAB,[1,D]),name="LAE")
    LAEB = tf.tensordot(tf.tensordot(LAE/tf.norm(LAE),B,[[1],[0]]),B,[[0,2],[0,2]],name="LAEB")
    LAEBE = tf.multiply(tf.multiply(LAEB,tf.reshape(EBA*EBA,[D,1])),tf.reshape(EBA*EBA,[1,D]),name="LAEBE")
    updateL = tf.assign(L,LAEBE/tf.norm(LAEBE))

In [92]:
with tf.name_scope("Energy"):
    LA = tf.tensordot(L,A,[[0],[0]])
    LAE = tf.multiply(LA,tf.reshape(EAB*EAB,[1,D,1]))
    LAEB = tf.tensordot(LAE,B,[[1],[0]])
    LAEBR = tf.tensordot(LAEB,R,[[2],[0]])
    LAEBRA = tf.tensordot(LAEBR,A,[[0],[0]])
    LAEBRAE = tf.multiply(LAEBRA,tf.reshape(EAB*EAB,[1,1,1,D,1]))
    LAEBRAEB = tf.tensordot(LAEBRAE,B,[[3,2],[0,1]])
    E_AB = tf.tensordot(LAEBRAEB,H,[[0,1,2,3],[0,1,2,3]])/tf.tensordot(LAEBRAEB,I,[[0,1,2,3],[0,1,2,3]])

In [93]:
E = E_AB
tf.summary.scalar("E",E)
merged = tf.summary.merge_all()

In [94]:
sess = tf.Session()
sess.run(init)

#if 'NotFirst' not in globals():
#    logger = tf.summary.FileWriter("/tmp/data/",sess.graph)
#    NotFirst = True

In [110]:
for i in xrange(1000):
    sess.run(updateAB)
    sess.run(updateBA)
for i in xrange(1000):
    sess.run(updateL)
    sess.run(updateR)

#logger.flush()
print sess.run(E)

-0.249915


In [111]:
sess.run(EAB)

array([ 0.50003141,  0.49999708,  0.49999672,  0.49997491], dtype=float32)

In [106]:
sess.run(B)

array([[[  2.47970708e-02,   1.12293929e-01],
        [ -5.85012458e-05,  -4.53873813e-01],
        [  5.37788355e-06,   1.52556285e-01],
        [ -7.82320858e-05,   8.67399797e-02]],

       [[  1.31419001e-05,  -2.63868928e-01],
        [ -6.91034645e-03,  -6.01418614e-02],
        [ -1.39917973e-02,  -1.95644140e-01],
        [ -1.05149556e-04,   3.71883035e-01]],

       [[  1.58813680e-04,  -2.98597157e-01],
        [  1.39360819e-02,  -1.85909241e-01],
        [ -6.89862669e-03,  -1.50298610e-01],
        [  1.24841303e-04,  -3.21630001e-01]],

       [[  3.48392496e-05,   2.79422075e-01],
        [ -9.55002834e-05,  -7.28579015e-02],
        [ -3.98652337e-05,  -4.07154202e-01],
        [  1.69845112e-03,  -2.74454392e-02]]], dtype=float32)